### `rbr_zip2depth`
Read RBR D|Fast .zip file and covert to depth

In [14]:
import zipfile
import pandas as pd

def read_rbr_zip(zip_path):
    """
    Read a Ruskin-exported RBR time series from a .zip file
    containing a file ending with '_data.txt'.

    Returns
    -------
    df : pandas.DataFrame
        Columns: Time, Pressure (dbar)
        indexed by Time.
    """
    with zipfile.ZipFile(zip_path, 'r') as zf:
        # Find the file ending with '_data.txt'
        member = [n for n in zf.namelist() if n.endswith('_data.txt')]
        if not member:
            raise FileNotFoundError("No file ending with '_data.txt' found in zip.")
        member = member[0]

        # Pandas can read directly from the binary file handle
        with zf.open(member) as f:
            df = pd.read_csv(f)

    # Parse and set datetime index
    df['Time'] = pd.to_datetime(df['Time'])
    df = df.set_index('Time')
    
    # remove the calculated columns
    df = df.drop(['Sea pressure', 'Depth'], axis=1)
    
    # rename the column to add units
    df.rename(columns={'Pressure': 'Pressure (dbar)'}, inplace=True)
    return df

                         Pressure (dbar)
Time                                    
2025-11-05 16:23:22.000        10.030985
2025-11-05 16:23:22.500        10.032069
2025-11-05 16:23:23.000        10.031802
2025-11-05 16:23:23.500        10.032199
2025-11-05 16:23:24.000        10.032729

Columns: ['Pressure (dbar)']


In [12]:
df

,Pressure,Sea pressure,Depth
Time,,,
2025-11-05 16:23:22.000,10.030985,-0.101515,-0.100687
2025-11-05 16:23:22.500,10.032069,-0.100431,-0.099612
2025-11-05 16:23:23.000,10.031802,-0.100698,-0.099877
2025-11-05 16:23:23.500,10.032199,-0.100301,-0.099483
2025-11-05 16:23:24.000,10.032729,-0.099771,-0.098958
...,...,...,...
2025-11-05 16:45:19.500,10.019546,-0.112954,-0.112033
2025-11-05 16:45:20.000,10.021326,-0.111174,-0.110267
2025-11-05 16:45:20.500,10.021632,-0.110868,-0.109964


In [ ]:
    zip_file = "C:/crs/proj/FA_science/202774_20251105_1646.zip"

### Hydrostatic relationship: calculating depth from pressure measurements

In a **static** water column, pressure increases with depth due to the weight of overlying water.  
The formaula for the hydrostatic relationship is:

$$
P = P_{atm} + \rho g z
$$

where:

- $P$ — measured (absolute) pressure (Pa)  
- $P_{atm}$ — atmospheric pressure at the surface (Pa)  
- $\rho$ — water density (kg m$^{-3}$)  
- $g$ — gravitational acceleration (m s$^{-2}$)  
- $z$ — depth below the free surface, positive downward (m)

(The alternative is a **dynamic** situation, which is more complicated. The **static** approximation is very close.)

---

#### Solve for depth by removing the atmospheric pressure and dividing by density and gravitational acceleration

$$
z = \frac{P - P_{atm}}{\rho g}
$$

---

#### Note on units

Force is mass * accleration, with units of kg * m / s$^2$, which are also called Newtons ($N$)  
  
Pressure is force per unit area, with units of N/m$^2$ = kg / (m s$^2$), which are also called Pascals ($Pa$)

Pressure sensors often report **absolute pressure** in decibar (dbar), where  
$1\ \text{dbar} = 10^{4}\ \text{Pa}$.

Decibars are handy units because 1 decibar equals about 1 meter of water depth.
Atmospheric pressure is about 10 decibars.

---

#### Typical parameters

- $P_{atm} \approx 101325\ \text{Pa}$  
- $\rho \approx 1025\ \text{kg m}^{-3}$ (seawater) or $1000\ \text{kg m}^{-3}$ (freshwater)  
- $g \approx 9.80665\ \text{m s}^{-2}$

---

#### Notes
  
- The relationship assumes **hydrostatic equilibrium** (dynamic pressure fluctuations are small).  
- Adjust $\rho$ and $g$ for local salinity, temperature, and latitude if needed.


In [ ]:
def calc_depth_from_pressure(
    df: pd.DataFrame,
    abs_col: str = "Pressure (dbar)",
    out_col: str = "Depth (m)",
    rho: float = 1025.,        # water density (kg m$^{-3}$)
    g: float = 9.80665,        # gravity (m s$^{-2}$)
    patm: float = 101592.0     # atmospheric pressure (Pa) = 30" mercury
) -> pd.DataFrame:
    """
    Compute depth from absolute pressure using the hydrostatic relation:

        z = (P_abs * 1e4 - p_atm) / (rho * g)

    Parameters
    ----------
    df : pandas.DataFrame
        Must contain a column with absolute pressure in dbar.
    abs_col : str
        Column name for absolute pressure (dbar).
    out_col : str
        Name of new depth column (m).
    rho : float
        Water density (kg m$^{-3}$).
    g : float
        Gravitational acceleration (m s$^{-2}$).
    patm : float
        Atmospheric pressure (Pa).

    Returns
    -------
    pandas.DataFrame
        The same DataFrame with a new column `out_col` (m).
    """
    if abs_col not in df.columns:
        raise KeyError(f"Column '{abs_col}' not found in DataFrame columns: {list(df.columns)}")

    # Convert absolute pressure (dbar) to Pa, subtract atmospheric pressure, and compute depth
    Psea_Pa = df[abs_col] * 1e4 - patm
    df[out_col] = Psea_Pa / (rho * g)

    return df


In [15]:
# Example usage
zip_file = "C:/crs/proj/FA_science/202774_20251105_1646.zip"
df = read_rbr_zip(zip_file)

print(df.head())
print("\nColumns:", df.columns.tolist())

                         Pressure (dbar)
Time                                    
2025-11-05 16:23:22.000        10.030985
2025-11-05 16:23:22.500        10.032069
2025-11-05 16:23:23.000        10.031802
2025-11-05 16:23:23.500        10.032199
2025-11-05 16:23:24.000        10.032729

Columns: ['Pressure (dbar)']
